In [4]:
import pandas as pd
column_names = ['Company Name', 'Email', 'Telephone']


url_csv_filename = lambda abbrv: "yellowpages_urls_" + abbrv + ".csv"
data_csv_filename = lambda abbrv: "yellowpages_data_" + abbrv + ".csv"
cleaned_csv_filename = lambda abbrv: "cleaned_" + abbrv + ".csv"

tasks = [
    {'category':0}
]

def clean_data(category):
    df_records = pd.read_csv('Category_Tracking.csv')
    category_data = df_records.loc[category]
    
    # Load the DataFrame from the CSV file
    df = pd.read_csv(data_csv_filename(category_data['Abbrv']), header=None, names=column_names)  

    # Save the corrected DataFrame back to the CSV file
    df.to_csv(cleaned_csv_filename(category_data['Abbrv']), index=False)

    # Iterate through each row and swap data if necessary
    for index, row in df.iterrows():
        # Check if the 'Email' column starts with 'Tel:' OR if the 'Telephone' column starts with 'Email:'
        if row['Email'].startswith('Tel:') or row['Telephone'].startswith('Email:'):
            # Swap the 'Email' and 'Telephone' data
            df.at[index, 'Email'], df.at[index, 'Telephone'] = row['Telephone'], row['Email']

    # Filter rows where the 'Email' column starts with 'Tel:'
    filtered_rows = df[df['Telephone'].str.startswith('Email:')]

    # Check if there are any such rows
    if not filtered_rows.empty:
        print("Rows where 'Telephone' starts with 'Email:':")
        print(filtered_rows)
    else:
        print("No rows found where 'Telephone' starts with 'Email:'.")

    # Filter rows where both 'Email' and 'Telephone' are 'Not Found'
    filtered_rows1 = df[(df['Email'] == 'Not found') & (df['Telephone'] == 'Not found')]
    print(f"Number of rows where both Email and Telephone are 'Not Found': {len(filtered_rows1)}")

    # Filter rows where both 'Email' and 'Telephone' are 'Not Found'
    filtered_rows2 = df[(df['Email'] == 'Not found')] 
    print(f"Number of rows where Email 'Not found': {len(filtered_rows2)}")

    # Filter rows where both 'Email' are 'Not Found'
    filtered_rows3 = df[(df['Telephone'] == 'Not found')] 
    print(f"Number of rows where Telephone are 'Not found': {len(filtered_rows3)}")
    
    
    ##Filter via Telephone
    
    # Remove rows where 'Telephone' & 'Email' is equal to 'Not found' MAIN FACTOR
    mask_factor = (df['Telephone'] != 'Not found') | (df['Email'] != 'Not found')
    output_df = df[mask_factor].copy()  # Use .copy() to avoid SettingWithCopyWarning

    # Optionally, reset the index of the DataFrame after removing rows
    output_df.reset_index(drop=True, inplace=True)

    # Cleaning the 'Company Name' column
    output_df['Company Name'] = output_df['Company Name'].str.replace(r"\(.*", "", regex=True)

    # Cleaning the 'Email' column
    output_df['Email'] = output_df['Email'].str.replace("Email: ", "", regex=False)

    # Cleaning the 'Telephone' column
    output_df['Telephone'] = output_df['Telephone'].str.replace("Tel: ", "", regex=False)

    # Identify rows where 'email' starts with 'fax:'
    mask = output_df['Email'].str.startswith('Fax:')
    output_df = output_df[~mask]

    # Remove duplicates
    output_df = output_df.drop_duplicates()

    # Sorting logic (if 'Email' is 'Not found', move to bottom)
    output_df['sort'] = output_df['Email'].apply(lambda x: 1 if x == 'Not found' else 0)
    final_df = output_df.sort_values(by='sort').drop('sort', axis=1)
    
    # Save the cleaned DataFrame to the same CSV file
    final_df.to_csv(data_csv_filename(category_data['Abbrv']), index=False)
    
    df_records.loc[category,'Data last search index'] = category_data['Amount of Data']
    df_records.loc[category,'Amount of Cleaned Email Data'] = len(pd.read_csv(cleaned_csv_filename(category_data['Abbrv'])))
    df_records.to_csv('Category_Tracking.csv', index=False)

for task in tasks:
    clean_data(task['category'])

df_records = pd.read_csv('Category_Tracking.csv')
print(df_records)

No rows found where 'Telephone' starts with 'Email:'.
Number of rows where both Email and Telephone are 'Not Found': 0
Number of rows where Email 'Not found': 16273
Number of rows where Telephone are 'Not found': 0
                        Category     Abbrv  Amount of URLS  \
0  Business Support and Supplies       bss           24473   
1            Cars and Automotive      cars           11698   
2   Construction and Engineering        CE           24827   
3       Computer and Electronics  computer            9187   
4        Fashion and Accessories       F&A            7020   
5      Healthcare and Medication       H&M           13836   
6               Home and Living        H&L           16913   
7                 Retail Stores     retail           12001   
8             Sports and Hobbies    sports            1655   
9      Trading and Manufacturing   trading           12001   

   URLS last search index  Amount of Data  Data last search index  \
0                   24473        

In [10]:
##Filter via Telephone
# Remove rows where 'Telephone' is equal to 'Not found' MAIN FACTOR
output_df = df[df['Telephone'] != 'Not found'].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Optionally, reset the index of the DataFrame after removing rows
output_df.reset_index(drop=True, inplace=True)

# Cleaning the 'Company Name' column
output_df['Company Name'] = output_df['Company Name'].str.replace(r"\(.*", "", regex=True)

# Cleaning the 'Email' column
output_df['Email'] = output_df['Email'].str.replace("Email: ", "", regex=False)

# Cleaning the 'Telephone' column
output_df['Telephone'] = output_df['Telephone'].str.replace("Tel: ", "", regex=False)

# Identify rows where 'email' starts with 'fax:'
mask = output_df['Email'].str.startswith('Fax:')
output_df = output_df[~mask]

# Remove duplicates
output_df = output_df.drop_duplicates()

# Sorting logic (if 'Email' is 'Not found', move to bottom)
output_df['sort'] = output_df['Email'].apply(lambda x: 1 if x == 'Not found' else 0)
final_df = output_df.sort_values(by='sort').drop('sort', axis=1)

# Display the final DataFrame
#final_df = final_df.iloc[:3000]  # Uncomment if you need to limit the rows
final_df

,Company Name,Email,Telephone
1,Pan Asiatic Enterprise Sdn Bhd,panasiatic@live.com,+603- 7729 5844
20942,Solid Granite & Marble Sdn Bhd,solid_mar@yahoo.com,+603-6140 2766
10724,Liew Sewing Machine & Trdg,liewsewingmachine@gmail.com,+607-255 5586
15754,Tasek Iron & Steel Foundry Sdn Bhd,myau@myjaring.net,+605-547 3493
10730,Setaracita Trdg Sdn Bhd,setaracita@gmail.com,+606-762 7409
...,...,...,...
9061,Tawakkal Bina Enterprise,Not found,+6096231366
9060,OKW Constn,Not found,+607-776 5996
9059,Bumita Sdn Bhd,Not found,+604-976 7226
8932,CCS Realty Sdn Bhd,Not found,+607-663 2208


In [9]:
# Save the cleaned DataFrame to the same CSV file
final_df.to_csv('28022024_C&E_Data_3000.csv', index=False)